In [13]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in /Users/Madison/opt/anaconda3/lib/python3.8/site-packages (0.0)


In [14]:
import pandas as pd
import numpy as np

# Read the CSV and Perform Basic Data Cleaning

In [15]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [16]:
df.columns

Index(['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag'],
      dtype='object')

# Select your features (columns)

In [65]:
# Set features. This will also be used as your x values.
# Upon review of the exoplanet archive website the following variables appear to have impact
# on if a exoplanet is confirmed. koi_time0bk was removed as it is a time it 
# was observed not a feature of the object. 
# Error was added back in and accuracy increase by ~0.05.
selected_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag']]
selected_features.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_impact,koi_impact_err1,koi_impact_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,0.586,0.059,-0.443,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,0.969,5.126,-0.077,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,1.276,0.115,-0.092,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,0.701,0.235,-0.478,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,0.762,0.139,-0.532,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Create a Train Test Split

Use `koi_disposition` for the y values

In [66]:
y = df["koi_disposition"]
y_binary_encoded = pd.get_dummies(y)
y_binary_encoded.head()

,CANDIDATE,CONFIRMED,FALSE POSITIVE
0,0,1,0
1,0,0,1
2,0,0,1
3,0,1,0
4,0,1,0


In [67]:
# setting y to be a single output as it is either confirmed or anything else and it has to be
# confirmed to be on the Exoplanet Archive Confirmed Planet table
y_confirmed = y_binary_encoded["CONFIRMED"]
y_confirmed.head()

0    1
1    0
2    0
3    1
4    1
Name: CONFIRMED, dtype: uint8

In [68]:
from sklearn.model_selection import train_test_split

#spliting training data, setting back to x and y for ease of modeling
X_train, X_test, y_train, y_test = train_test_split(selected_features, y_confirmed, random_state=57)

In [69]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_impact,koi_impact_err1,koi_impact_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
6279,1,0,0,0,8.808542,0.000061,-0.000061,0.9930,0.1160,-0.1360,...,-181,4.300,0.135,-0.135,1.210,0.245,-0.200,295.71942,45.747559,12.239
2166,0,0,0,0,12.924898,0.000023,-0.000023,0.7680,0.1500,-0.5240,...,-102,4.584,0.020,-0.056,0.768,0.054,-0.031,298.66092,48.606369,13.524
1815,0,0,0,0,6.124798,0.000032,-0.000032,0.7900,0.0370,-0.5450,...,-133,4.285,0.137,-0.112,1.202,0.208,-0.189,288.37024,43.354610,13.268
620,0,0,0,0,7.886632,0.000003,-0.000003,0.0480,0.1000,-0.0480,...,-193,4.304,0.180,-0.180,1.135,0.333,-0.222,295.09500,43.537579,15.014
6442,0,0,1,0,4.080293,0.000011,-0.000011,0.2428,0.2128,-0.2427,...,-156,4.536,0.030,-0.291,0.874,0.366,-0.067,296.38229,48.045311,15.488


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [70]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
# only X variable needs to be scaled since y is all binary encoding
# using training data to create the scaler 
X_MinMaxScaler = MinMaxScaler().fit(X_train)

In [71]:
X_train_scaled = X_MinMaxScaler.transform(X_train)
X_test_scaled = X_MinMaxScaler.transform(X_test)
X_train_scaled

array([[1.        , 0.        , 0.        , ..., 0.72552354, 0.58191471,
        0.43582114],
       [0.        , 0.        , 0.        , ..., 0.86005558, 0.76332706,
        0.54202827],
       [0.        , 0.        , 0.        , ..., 0.38940245, 0.43006463,
        0.52086949],
       ...,
       [0.        , 0.        , 0.        , ..., 0.6968284 , 0.8479607 ,
        0.63542442],
       [0.        , 0.        , 0.        , ..., 0.80304628, 0.60179353,
        0.67476651],
       [0.        , 0.        , 0.        , ..., 0.64636802, 0.6434976 ,
        0.67617158]])

# Train the Model



In [72]:
# first model tested is linear regression to get a base comparison 
from sklearn.linear_model import LogisticRegression
model2 = LogisticRegression()
model2.fit(X_train_scaled, y_train)

/Users/Madison/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [73]:
print(f"Training Data Score: {model2.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model2.score(X_test_scaled, y_test)}")

Training Data Score: 0.8430288003051688
Testing Data Score: 0.830091533180778


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [79]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
# oringally ran C with 1, 5, 10 and 10 was the best so added C options
# next ran with 1, 5, 10, 20, 50 and 50 was the best so tried another set (see final set below)
param_grid2 = {'C': [10, 50, 100, 1000, 5000, 10000],
              'fit_intercept': [True, False],
              'max_iter': [100, 200, 300]}
grid2 = GridSearchCV(model2, param_grid2, verbose=3)

In [80]:
# Train the model with GridSearch
grid2.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV] C=10, fit_intercept=True, max_iter=100 ..........................
[CV]  C=10, fit_intercept=True, max_iter=100, score=0.846, total=   0.1s
[CV] C=10, fit_intercept=True, max_iter=100 ..........................
[CV]  C=10, fit_intercept=True, max_iter=100, score=0.878, total=   0.1s
[CV] C=10, fit_intercept=True, max_iter=100 ..........................
[CV]  C=10, fit_intercept=True, max_iter=100, score=0.868, total=   0.1s
[CV] C=10, fit_intercept=True, max_iter=100 ..........................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/Madison/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
/Users/Madison/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/

[CV]  C=10, fit_intercept=True, max_iter=100, score=0.864, total=   0.1s
[CV] C=10, fit_intercept=True, max_iter=100 ..........................
[CV]  C=10, fit_intercept=True, max_iter=100, score=0.863, total=   0.1s
[CV] C=10, fit_intercept=True, max_iter=200 ..........................
[CV]  C=10, fit_intercept=True, max_iter=200, score=0.847, total=   0.1s
[CV] C=10, fit_intercept=True, max_iter=200 ..........................


/Users/Madison/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/Madison/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

[CV]  C=10, fit_intercept=True, max_iter=200, score=0.875, total=   0.2s
[CV] C=10, fit_intercept=True, max_iter=200 ..........................
[CV]  C=10, fit_intercept=True, max_iter=200, score=0.867, total=   0.1s
[CV] C=10, fit_intercept=True, max_iter=200 ..........................


...

[CV]  C=10000, fit_intercept=False, max_iter=300, score=0.874, total=   0.2s
[CV] C=10000, fit_intercept=False, max_iter=300 ......................
[CV]  C=10000, fit_intercept=False, max_iter=300, score=0.887, total=   0.2s
[CV] C=10000, fit_intercept=False, max_iter=300 ......................


/Users/Madison/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/Madison/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

[CV]  C=10000, fit_intercept=False, max_iter=300, score=0.882, total=   0.2s
[CV] C=10000, fit_intercept=False, max_iter=300 ......................
[CV]  C=10000, fit_intercept=False, max_iter=300, score=0.882, total=   0.2s
[CV] C=10000, fit_intercept=False, max_iter=300 ......................
[CV]  C=10000, fit_intercept=False, max_iter=300, score=0.876, total=   0.2s


/Users/Madison/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done 180 out of 180 | elapsed:   22.1s finished
/Users/Madison/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver op

GridSearchCV(estimator=LogisticRegression(),
             param_grid={'C': [10, 50, 100, 1000, 5000, 10000],
                         'fit_intercept': [True, False],
                         'max_iter': [100, 200, 300]},
             verbose=3)

In [81]:
print(grid2.best_params_)
print(grid2.best_score_)

{'C': 5000, 'fit_intercept': True, 'max_iter': 300}
0.8806029370028889


# Save the Model

In [82]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'mbfisher_model2.sav'
joblib.dump(grid2, filename)

['mbfisher_model2.sav']

In [83]:
predict_values = grid2.predict(X_test_scaled)
pd.DataFrame({"Prediction of Status": predict_values, "Actual Status": y_test})

,Prediction of Status,Actual Status
1744,1,0
4323,0,0
2919,0,1
3391,0,1
3691,0,0
...,...,...
5459,0,0
4007,0,0
1761,0,0
2068,0,0
